In [1]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.3 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


In [2]:
import torch
import numpy as np
import pandas as pd
from datasets import load_metric
from torch.utils.data import DataLoader
from datasets import Dataset, ClassLabel, Sequence, Features, Value, DatasetDict
from transformers import AutoModel,AutoTokenizer,AutoModelForSequenceClassification,AutoModelForTokenClassification, AdamW, DataCollatorForTokenClassification

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
path="/kaggle/input/151s5d1fs6e15fa/"
df = pd.read_json(path+"train.json",lines=True)
test_df = pd.read_json(path+"test.json",lines=True)
valid_df = pd.read_json(path+"valid.json",lines=True)
print(len(df))
print(len(test_df))
print(len(valid_df))
df[:2]

5228
5865
5330


,tags,tokens
0,"[1, 0, 0, 0, 0, 0, 1, 0]","[Naloxone, reverses, the, antihypertensive, ef..."
1,"[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[In, unanesthetized, ,, spontaneously, hyperte..."


In [4]:
tag_name = ["O",
    "B-Chemical",
    "B-Disease",
    "I-Disease",
    "I-Chemical"]

In [5]:
tags = ClassLabel(num_classes=len(tag_name), names=tag_name)

In [6]:
tags

ClassLabel(num_classes=5, names=['O', 'B-Chemical', 'B-Disease', 'I-Disease', 'I-Chemical'], id=None)

In [7]:
dataset_structure = {"ner_tags":Sequence(tags),
                 'tokens': Sequence(feature=Value(dtype='string'))}

In [8]:
dataset_structure

{'ner_tags': Sequence(feature=ClassLabel(num_classes=5, names=['O', 'B-Chemical', 'B-Disease', 'I-Disease', 'I-Chemical'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [9]:
dataset_structure["ner_tags"].feature.names

['O', 'B-Chemical', 'B-Disease', 'I-Disease', 'I-Chemical']

In [10]:
def df_to_dataset(df, columns=['tags', 'tokens']):
  ner_tags = df['tags']
  tokens = df['tokens']
  d = {'ner_tags':ner_tags, 'tokens':tokens}
  dataset = Dataset.from_dict(mapping=d,features=Features(dataset_structure),)
  return dataset

dataset = df_to_dataset(df)
test_dataset =  df_to_dataset(test_df)
valid_dataset =  df_to_dataset(valid_df)

dataset = DatasetDict({
    'train': dataset,
    'test': test_dataset,
    'valid': valid_dataset})

label_names = dataset['train'].features["ner_tags"].feature.names
label_names

['O', 'B-Chemical', 'B-Disease', 'I-Disease', 'I-Chemical']

In [11]:
dataset['train']

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 5228
})

In [12]:
dataset['train'][:1]

{'ner_tags': [[1, 0, 0, 0, 0, 0, 1, 0]],
 'tokens': [['Naloxone',
   'reverses',
   'the',
   'antihypertensive',
   'effect',
   'of',
   'clonidine',
   '.']]}

In [13]:
# model_name="xlm-roberta-xlarge"
model_name= 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
# def tokenize_function(sample):
#     return tokenizer(sample["tokens"], padding="max_length",truncation=True, is_split_into_words=True)

In [15]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [16]:
# tokenized_datasets['train'][0]['input_ids'][:20]

In [17]:
# tokenized_datasets['train'][0]['ner_tags'][:20]

In [18]:
# len(tokenized_datasets['train'][0]['input_ids']) == len(tokenized_datasets['train'][0]['ner_tags'])

In [19]:
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"],
                is_split_into_words=True, truncation=True)

  total_adjusted_labels = []

  for k in range(0, len(tokenized_samples["input_ids"])):
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    adjusted_label_ids = []
    prev_wid = -1
    i = -1
    for word_idx in word_ids_list:
      if(word_idx is None):
        adjusted_label_ids.append(-100)
      elif(word_idx!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = word_idx
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)

  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels,batched=True,remove_columns=list(dataset["train"].features.keys()))

  0%|          | 0/6 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [20]:
# out = tokenizer("Fine tune NER in google colab!")
# out

In [21]:
# out.word_ids(0)

In [22]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5228
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5865
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5330
    })
})

In [23]:
tokenized_dataset['train'][0]['input_ids']

[1, 7460, 43623, 2268, 57375, 262, 97424, 1290, 265, 114354, 323, 2]

In [24]:
tokenized_dataset['train'][0]['attention_mask']

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [25]:
tokenized_dataset['train'][0]['labels']

[-100, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, -100]

In [26]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [28]:
# model = AutoModelForTokenClassification.from_pretrained(model_name,num_labels=len(label_names))
model = AutoModelForTokenClassification.from_pretrained(model_name,num_labels=len(label_names))
model.to(device)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForTokenClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a B

DebertaV2ForTokenClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwis

In [29]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [30]:
example = dataset["train"][1]
labels = [label_names[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'Chemical': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Disease': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [31]:
from transformers import TrainingArguments, Trainer

epochs = 8
batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/bert-fine-tune-ner/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [32]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.128900,0.113483,0.864096,0.942336,0.901522,0.964058
2,0.046000,0.120513,0.918643,0.914677,0.916656,0.973143
3,0.024400,0.119551,0.925129,0.923897,0.924513,0.974144
4,0.013300,0.165880,0.936544,0.917129,0.926735,0.974712
5,0.006200,0.133692,0.914872,0.939299,0.926924,0.974488
6,0.004400,0.162807,0.912343,0.939085,0.925521,0.972983
7,0.001100,0.177003,0.929980,0.930079,0.930029,0.975825
8,0.000400,0.179484,0.928189,0.933383,0.930779,0.975953


TrainOutput(global_step=5232, training_loss=0.028046699950142834, metrics={'train_runtime': 2525.8298, 'train_samples_per_second': 16.559, 'train_steps_per_second': 2.071, 'total_flos': 4102930141203768.0, 'train_loss': 0.028046699950142834, 'epoch': 8.0})

In [33]:
trainer.evaluate()

{'eval_loss': 0.17948400974273682,
 'eval_precision': 0.9281890932216864,
 'eval_recall': 0.9333830739714346,
 'eval_f1': 0.9307788377222119,
 'eval_accuracy': 0.9759531871632926,
 'eval_runtime': 54.7615,
 'eval_samples_per_second': 97.331,
 'eval_steps_per_second': 12.18,
 'epoch': 8.0}

In [34]:
# torch.save(model, '123')
# model = torch.load('123')
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["valid"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )
# trainer.evaluate()

In [35]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results["overall_f1"]

0.9243355914538822